In [3]:
import pandas as pd
import os
from main import list_files_in_folder
from dotenv import load_dotenv
from dataClass import DataTable
from langchain_mistralai import ChatMistralAI
import json
from tqdm import tqdm
from utils import get_table_str, candidates_as_str, build_prompt
from prompts import generate_CEA_prompt_with_t_desc
import time

In [4]:
prompts = pd.read_json(path_or_buf='HT_Prompts_Valid.jsonl', lines=True)
prompt = prompts.iloc[5]['prompt_desc']

In [12]:
prompt_desc = prompts[prompts['table']=='DFRU6OJ0'].iloc[0]['prompt_desc']
prompt_nodesc = prompts[prompts['table']=='DFRU6OJ0'].iloc[0]['prompt_nodesc']


In [13]:
mistral_api_key = os.getenv("MISTRAL_API_KEY")
model_22 = "open-mixtral-8x22b"
model_7 = "open-mixtral-8x7b"
llm_22 = ChatMistralAI(model=model_22, temperature=0, api_key=mistral_api_key)
llm_7 = ChatMistralAI(model=model_7, temperature=0, api_key=mistral_api_key)

In [14]:
out_desc7 = llm_7.invoke(prompt_desc)
out_nodesc7 = llm_7.invoke(prompt_nodesc)
out_desc22 = llm_22.invoke(prompt_desc)
out_nodesc22 = llm_22.invoke(prompt_nodesc)


LLM 7 Response (Prompt no desc): content="The correct referent entity for the entity mention 'Predlitz' is '<Predlitz [DESC] former municipality in Austria [TYPE] rural municipality of Austria>'." response_metadata={'token_usage': {'prompt_tokens': 1448, 'total_tokens': 1485, 'completion_tokens': 37}, 'model': 'open-mixtral-8x7b', 'finish_reason': 'stop'} id='run-38a96a76-bd79-412c-a721-d9e0a064e08e-0'
LLM 7 Response (Prompt desc): content="The correct referent entity for the entity mention 'Predlitz' is '<Predlitz [DESC] former municipality in Austria [TYPE] rural municipality of Austria>'. The reason for this is that, according to the input, column 0 contains German place names, specifically the names of small towns or villages. 'Predlitz' appears to be a unique name within the column and starts with 'pred-', which suggests that it is a name of a small town or village in Germany or Austria. The referent entity candidate that matches this description is '<Predlitz [DESC] former munici

In [15]:
print(f"LLM 7 Response (Prompt no desc): {out_nodesc7.content}")
print(f"LLM 7 Response (Prompt desc): {out_desc7.content}")
print(f"LLM 22 Response (Prompt no desc): {out_nodesc22.content}")
print(f"LLM 22 Response (Prompt desc): {out_desc22.content}")

LLM 7 Response (Prompt no desc): The correct referent entity for the entity mention 'Predlitz' is '<Predlitz [DESC] former municipality in Austria [TYPE] rural municipality of Austria>'.
LLM 7 Response (Prompt desc): The correct referent entity for the entity mention 'Predlitz' is '<Predlitz [DESC] former municipality in Austria [TYPE] rural municipality of Austria>'. The reason for this is that, according to the input, column 0 contains German place names, specifically the names of small towns or villages. 'Predlitz' appears to be a unique name within the column and starts with 'pred-', which suggests that it is a name of a small town or village in Germany or Austria. The referent entity candidate that matches this description is '<Predlitz [DESC] former municipality in Austria [TYPE] rural municipality of Austria>'.
LLM 22 Response (Prompt no desc): The correct referent entity for the entity mention 'Predlitz' is <Predlitz [DESC] former municipality in Austria [TYPE] rural municipali

In [32]:
from mistralai.client import MistralClient

api_key = os.environ["MISTRAL_API_KEY"]
client = MistralClient(api_key=api_key)

embeddings_batch_response = client.embeddings(
      model="mistral-embed",
      input=["Embed this sentence.", "As well as this one."],
  )